<a href="https://colab.research.google.com/github/Cinnamy/DocumentsClassification/blob/main/3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключаем модули

In [ ]:
import sklearn
import numpy as np
import nltk
import re
import pandas as pd
import torch

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


#Загружаем и предобрабатываем данные

In [ ]:
stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

Датасет с хорошо разделёнными тематиками:

In [ ]:
well_separated = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=['comp.graphics', 'rec.autos', 'sci.med', 'talk.politics.mideast'])
well_separated_full = []
well_separated_target = [[], [], []]
well_separated_nouns = []
well_separated_nouns_and_adjs = []
for i in range(300):
  text = well_separated.data[i]
  text = re.sub(r'[^(\w\s\-)]|[\d]|[()]|-{2,}', '', text)
  text = re.sub(r'\s{2,}|[\n]', ' ', text)
  text = ' '.join([lemmatizer.lemmatize(word.lower()) for word in word_tokenize(text) if word not in stopwords_en])
  if text != '':
    well_separated_full.append(text)
    well_separated_target[0].append(well_separated.target[i])
  nouns = ' '.join([word[0] for word in nltk.pos_tag(word_tokenize(text)) if word[1] == "NN"])
  if nouns != '':
    well_separated_nouns.append(nouns)
    well_separated_target[1].append(well_separated.target[i])
  nouns_and_adjs = ' '.join([word[0] for word in nltk.pos_tag(word_tokenize(text)) if word[1] == "NN" or word[1] == "JJ"])
  if nouns_and_adjs != '':
    well_separated_nouns_and_adjs.append(nouns_and_adjs)
    well_separated_target[2].append(well_separated.target[i])

Датасет с плохо разделёнными тематиками:

In [ ]:
weak_separated = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=['talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc'])
weak_separated_full = []
weak_separated_target = [[], [], []]
weak_separated_nouns = []
weak_separated_nouns_and_adjs = []
for i in range(300):
  text = weak_separated.data[i]
  text = re.sub(r'[^(\w\s\-)]|[\d]|-{2,}', '', text)
  text = re.sub(r'\s{2,}|[\n]', ' ', text)
  text = ' '.join([lemmatizer.lemmatize(word.lower()) for word in word_tokenize(text) if word not in stopwords_en])
  if text != '':
    weak_separated_full.append(text)
    weak_separated_target[0].append(weak_separated.target[i])
  nouns = ' '.join([word[0] for word in nltk.pos_tag(word_tokenize(text)) if word[1] == "NN"])
  if nouns != '':
    weak_separated_nouns.append(nouns)
    weak_separated_target[1].append(weak_separated.target[i])
  nouns_and_adjs = ' '.join([word[0] for word in nltk.pos_tag(word_tokenize(text)) if word[1] == "NN" or word[1] == "JJ"])
  if nouns_and_adjs != '':
    weak_separated_nouns_and_adjs.append(nouns_and_adjs)
    weak_separated_target[2].append(weak_separated.target[i])

#3 задание

##Хорошо разделённые тематики

In [ ]:
vectorizer = TfidfVectorizer()
X_full = vectorizer.fit_transform(well_separated_full)
X_nouns = vectorizer.fit_transform(well_separated_nouns)
X_nouns_and_adjs = vectorizer.fit_transform(well_separated_nouns_and_adjs)

###KMeans

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full, X_nouns, X_nouns_and_adjs]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=3, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X.toarray(), kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X.toarray(), kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(well_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans, well separated")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.162050,0.002448,1.695053,10.597641
Nouns,0.166836,0.002396,1.790666,10.325917
Nouns and adjs,0.216322,0.003357,1.936155,10.420666


###Agglomerative, Euclidean

In [ ]:
aggl_euc_metrics = [[], [], []]

for X in datasets:
  i = datasets.index(X)
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X.toarray())
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X.toarray(), aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X.toarray(), aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean, well separated")

###Agglomerative, cosine

In [ ]:
aggl_cos_metrics = [[], [], []]

datasets = [X_full, X_nouns, X_nouns_and_adjs]
for X in datasets:
  i = datasets.index(X)
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X.toarray())
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X.toarray(), aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X.toarray(), aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine, well separated")

##Плохо разделённые тематики

In [ ]:
vectorizer = TfidfVectorizer()
X_full = vectorizer.fit_transform(weak_separated_full)
X_nouns = vectorizer.fit_transform(weak_separated_nouns)
X_nouns_and_adjs = vectorizer.fit_transform(weak_separated_nouns_and_adjs)

###KMeans

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full, X_nouns, X_nouns_and_adjs]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=3).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X.toarray(), kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X.toarray(), kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(weak_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans, weak separated")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.080921,0.002618,1.791327,10.330237
Nouns,0.029290,0.001340,1.656025,10.508624
Nouns and adjs,0.137873,0.003782,2.230398,9.549711


###Agglomerative, Euclidean

In [ ]:
aggl_euc_metrics = [[], [], []]

for X in datasets:
  i = datasets.index(X)
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X.toarray())
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X.toarray(), aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X.toarray(), aggl.labels_)
      sum_v_measure += metrics.v_measure_score(weak_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean, weak separated")

###Agglomerative, cosine

In [ ]:
aggl_cos_metrics = [[], [], []]

datasets = [X_full, X_nouns, X_nouns_and_adjs]
for X in datasets:
  i = datasets.index(X)
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X.toarray())
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X.toarray(), aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X.toarray(), aggl.labels_)
      sum_v_measure += metrics.v_measure_score(weak_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine, weak separated")